<a href="https://colab.research.google.com/github/LIMDANBI/Hackathon/blob/main/XAI_using_pytorch_Captum_(%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경 설정

In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-k5mjas9a
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-k5mjas9a
     |████████████████████████████████| 129 kB 12.8 MB/s 
     |████████████████████████████████| 344 kB 81.0 MB/s 
     |████████████████████████████████| 54.7 MB 5.8 MB/s 
     |████████████████████████████████| 4.5 MB 57.8 MB/s 
     |████████████████████████████████| 1.2 MB 70.3 MB/s 
     |██████████████████████████████▎ | 834.1 MB 122.3 MB/s eta 0:00:01tcmalloc: large alloc 1147494400 bytes == 0x3a11c000 @  0x7f5168287615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118

In [2]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import gluonnlp as nlp
from tqdm.notebook import tqdm

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

In [3]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

# 데이터 불러오기

In [8]:
# -*- coding: cp949 -*-
df = pd.read_csv("/content/review.csv")
df

,Unnamed: 0,star,review
0,0,1,배공빠르고 굿
1,1,0,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,2,1,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,3,0,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,4,1,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...,...
393886,193886,1,할머니가 기침을 자주 하셔서 사드렸어요. 맛있다고 하시네요ㅎㅎ 아직 얼마 안돼서 잘...
393887,193887,1,아이들이 항상 이제품만 찾아요 색상도 좋고 애들착용도 좋네요
393888,193888,1,6개씩 박스 포장 되어 오네요22.04.05제조제조일이 신선하고황제펭귄 라벨 귀여워...
393889,193889,1,호두를 좋아하는 딸아이를 위해서 주문했어요^^


In [9]:
df['star'].value_counts()

1    199963
0    193928
Name: star, dtype: int64

# Input for BERTDataset

In [10]:
data_list = []

for review, label in zip(df['review'], df['star']):
  data = []
  data.append(review)
  data.append(label)
  data_list.append(data)

len(data_list)

393891

In [11]:
data_list[:5]

[['배공빠르고 굿', 1],
 ['택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고', 0],
 ['아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.', 1],
 ['선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요..',
  0],
 ['민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ', 1]]

# Train, Test Split

In [37]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_list, test_size = 0.2, shuffle=True, random_state = 0)

In [38]:
print(len(train))
train[:5]

315112


[['밑에는 봉지으로만 감싸있어서 놀랬답니다혹시나 하고 봉지를 꺼내서 보니역시나 밑에 4면이 바닥에 많이 내팽켜쳐질때많이 생기는 부분들이 있더군요플라스틱 바닥으로 된 제품을 시키는데종이를 윗면만 하지말고밑에도 해주셨으면 상처 안났을꺼라고 생각합니다더보기',
  0],
 ['재질은 보통이고 얇아요. 가격이 싸서 샀어요 요리할때나 드러운곳 청소할때 막 흐긴엔 편합니다', 1],
 ['전에 300g에 2500원정도 구매했었는데 몇그램인지 확인안하고 금액만 보고 구매했어요. 확인안하고 구매한 제탓이지만 너무 비싸요. 다른데는 800원씩인데요.',
  0],
 ['아직 사용전인데 십일절에 많이 할인하길래 아직 화장지가 남았지만 주문을 했습니다.그런데 오늘또 그 가격쯤에 할인을 하네요.원래 이 가격이 아닐까?하는 의심도 드네요.화장지 질이 좋아서 후회하는일만 없었으면 좋겠습니다',
  1],
 ['압력밥솥믿고 삽니다 좋아요', 1]]

In [39]:
print(len(test))
test[:5]

78779


[['배송최악. 처음시킨것도 아닌데... 주소를 못찾는게 내가 주소를 잘못입력했다고ㅋㅋ 하루에 택배가 두세개씩오는데 어이가없어서ㅋㅋ', 0],
 ['새차사서 구매했구요, 차량 트렁크 깔끔하게 잘쓰고있어요^^', 1],
 ['아니 아직 받지도않앗는데 무슨 구매평이고', 0],
 ['사용하기 편하네요. 다음에도 구매할께요', 1],
 ['아고 이건 또 막대길이가 불량인지 뒷 손잡이 걸이가 쑥~ 빠져서 걸수가 없군요. 길이 조절되는건 괜찮은데 스펀지도 고무날쪽도 쫌 별루네요.',
  0]]

# KoBERT tokenizer, model

[CLS] : classification token (문장의 시작)

[SEP] : seperate (문장 구분)

[MASK] : MASK

[PAD] : PADDING

In [40]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [41]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [42]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# Dataset

In [43]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# Parameter

In [44]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [45]:
# for i in train:
#   print(i[0])
#   print(i[1])
#   break

In [46]:
train_dataset = BERTDataset(train, 0, 1, tok, max_len, True, False)
test_dataset = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [47]:
train_dataset.labels[:3]

[0, 1, 0]

In [48]:
test_dataset.sentences[:3]

[(array([   2, 2287, 6621, 7458, 6811,  517,   54, 4468, 6726, 5398, 5859,
         3105, 5850,  517,   54,  517,   54,  517,   54, 4213, 6607, 6116,
         2086, 7404, 5760, 5400, 1435, 4213, 6607, 6116, 3943, 7138, 6064,
         7870,  492,  492, 4937, 6896,  517, 7601, 6312, 5330, 1773, 6579,
         5357, 6792, 6964, 5761, 3220, 7096, 5330, 6881, 6855, 6553,  492,
          492,    3,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
  array(57, dtype=int32),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=int32)),
 (array([   2, 2695, 7389, 6493, 6553, 1119, 7864, 5495, 6999,  517,   46,
         4406, 4773, 6048, 7565, 1340, 5646, 7784, 3942, 6779, 5439, 7141,
         6857,  517,  364,  517,  364,    3,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,   

# DataLoader

In [49]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=5)

# KoBERT 모델

In [50]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
        out = self.dropout(pooler)
    else:
        out = pooler
    return self.classifier(out)

In [51]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer and schedule

In [52]:
no_decay = ['bias', 'LayerNorm.weight']

# 최적화해야 할 parameter를 optimizer에게 알려야 함
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [53]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate) # optimizer
loss_fn = nn.CrossEntropyLoss() # loss function 

In [54]:
t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [55]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# Calculate Accuracy

In [56]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# Train

In [57]:
for e in range(num_epochs):
    
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()
        
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_loader), total=len(test_loader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/4924 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7832643389701843 train acc 0.40625
epoch 1 batch id 201 loss 0.6105517745018005 train acc 0.5562810945273632
epoch 1 batch id 401 loss 0.24744471907615662 train acc 0.7072552992518704
epoch 1 batch id 601 loss 0.22553612291812897 train acc 0.7717606073211315
epoch 1 batch id 801 loss 0.3537489175796509 train acc 0.8047167602996255
epoch 1 batch id 1001 loss 0.21686290204524994 train acc 0.8252060439560439
epoch 1 batch id 1201 loss 0.2335302233695984 train acc 0.8399771024146545
epoch 1 batch id 1401 loss 0.17764805257320404 train acc 0.8504528015703069
epoch 1 batch id 1601 loss 0.29788902401924133 train acc 0.8586137570268582
epoch 1 batch id 1801 loss 0.18149495124816895 train acc 0.8655868267629095
epoch 1 batch id 2001 loss 0.2661488652229309 train acc 0.871072276361819
epoch 1 batch id 2201 loss 0.22251826524734497 train acc 0.8750496933212176
epoch 1 batch id 2401 loss 0.13327926397323608 train acc 0.8786247917534361
epoch 1 batch id 2601 loss 0.1634678

  0%|          | 0/1231 [00:00<?, ?it/s]

epoch 1 test acc 0.9245351374795192


  0%|          | 0/4924 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9368229718845089


  0%|          | 0/4924 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.08986960351467133 train acc 0.96875
epoch 5 batch id 201 loss 0.08379698544740677 train acc 0.9744247512437811
epoch 5 batch id 401 loss 0.015500541776418686 train acc 0.9739323566084788
epoch 5 batch id 601 loss 0.08081381767988205 train acc 0.9743656405990017
epoch 5 batch id 801 loss 0.04962598904967308 train acc 0.9749726903870163
epoch 5 batch id 1001 loss 0.03815104067325592 train acc 0.974744005994006
epoch 5 batch id 1201 loss 0.09309203922748566 train acc 0.9752680058284763
epoch 5 batch id 1401 loss 0.10618125647306442 train acc 0.9752408993576017
epoch 5 batch id 1601 loss 0.13607268035411835 train acc 0.9751034509681449
epoch 5 batch id 1801 loss 0.1494244784116745 train acc 0.9754476679622432
epoch 5 batch id 2001 loss 0.06914922595024109 train acc 0.9759026736631684
epoch 5 batch id 2201 loss 0.009727205149829388 train acc 0.9758348477964561
epoch 5 batch id 2401 loss 0.010483759455382824 train acc 0.9759930758017493
epoch 5 batch id 2601 loss 0.

  0%|          | 0/1231 [00:00<?, ?it/s]

epoch 5 test acc 0.9370768305360118


# 모델 저장

In [58]:
# PATH = '/content/'
# # torch.save(model, PATH + 'naverShoppingReview.pt')  # 전체 모델 저장
# torch.save(model.state_dict(), PATH + 'naverShoppingReview_state_dict.pt')  # 모델 객체의 state_dict 저장
# # torch.save({
# #     'model': model.state_dict(),
# #     'optimizer': optimizer.state_dict()
# # }, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

# XAI using pytorch Captum

In [59]:
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
transform = nlp.data.BERTSentenceTransform(tok, max_seq_length = max_len, pad=True, pair=False)
transform(['너무 예뻐요'])

(array([   2, 1458, 3396, 6471, 6999,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(6, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32))

In [72]:
tok("너무 예뻐요")

['▁너무', '▁예', '뻐', '요']

In [74]:
tok.sentencepiece("너무 예뻐요")

['▁너무', '▁예', '뻐', '요']

In [71]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = tok.vocab.padding_token
PAD_IND = tok.convert_tokens_to_ids(PAD_IND)
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND) # 레퍼런스 생성을 위한 모듈
lig = LayerIntegratedGradients(model, model.bert.embeddings) # 결과 해석에 사용되는 IntegratedGradient 기법 모듈

In [69]:
vis_data_records_ig = []

label_vocab  = {0: '부정', 1: '긍정'}

In [75]:
def forward_with_sigmoid_for_bert(input,valid_length,segment_ids):
    return torch.sigmoid(model(input,valid_length,segment_ids))

In [76]:
def forward_for_bert(input,valid_length,segment_ids):
    return torch.nn.functional.softmax(model(input,valid_length,segment_ids),dim=1)

In [83]:
vis_data_records_ig = []
LABEL_COLUMNS = ["부정", "긍정"]
voc_label_dict_inverse={ele:LABEL_COLUMNS.index(ele) for ele in LABEL_COLUMNS}
voc_label_dict={LABEL_COLUMNS.index(ele):ele for ele in LABEL_COLUMNS}

In [84]:
def add_attributions_to_visualizer(attributions, input_text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            voc_label_dict[pred_ind], #Label.vocab.itos[pred_ind],
                            voc_label_dict[label], # Label.vocab.itos[label],
                            100, # Label.vocab.itos[1],
                            attributions.sum(),       
                            input_text,
                            delta))

In [85]:
def interpret_sentence(model, sentence, min_len = 64, label = 0, n_steps=10):      
    
    seq_tokens = transform([sentence])
    indexed = torch.tensor(seq_tokens[0]).long().to(device)
    valid_length = torch.tensor(seq_tokens[1]).long().unsqueeze(0)
    segment_ids = torch.tensor(seq_tokens[2]).long().unsqueeze(0).to(device)
    sentence = [token for token in tok.sentencepiece(sentence)]

    with torch.no_grad():
      model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid_for_bert(input_indices,valid_length,segment_ids).detach().cpu().numpy().argmax().item()
    print(forward_with_sigmoid_for_bert(input_indices,valid_length,segment_ids))
    pred_ind = round(pred)
    
    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices,\
                                            n_steps=n_steps, return_convergence_delta=True,target=label,\
                                            additional_forward_args=(valid_length,segment_ids))

    add_attributions_to_visualizer(attributions_ig, sentence, pred, pred_ind, label, delta, vis_data_records_ig)

In [86]:
# 예제 문장 추가 및 분석 수행
interpret_sentence(model, '옷이 너무 예뻐요!! 입고 나갔더니 다들 어디꺼나고 물어보네요 ㅎㅎ', label=1)
interpret_sentence(model, '박스가 다 터져서 왔네요;; 실망이에요 ㅜ', label=0)
interpret_sentence(model, '정말 가성비짱이네요! 사람들이 다 추천하는데는 역시 이유가 있네요', label=1)
interpret_sentence(model, '이렇게 장사하지 마세요 ㅠ 배송도 무슨 1달이나 걸리고,, 상품도 상태가 영,,', label=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


tensor([[0.0407, 0.9427]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9661, 0.0340]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.0427, 0.9410]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9687, 0.0336]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [87]:
# 시각화 결과 표 변환
visualization.visualize_text(vis_data_records_ig)